深度学习
=============

任务 2
------------

之前在 `1_notmnist.ipynb` 中, 我们使用格式化的数据集创建了一个 pickle 用来训练、开发和测试 [notMNIST 数据集](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html)。

这个任务的目的是利用 Tensorflow 进一步训练更深、更精确的模型。

In [0]:
# 这些是我们后面会使用的模块。确保在进一步操作之前你可以
# 将所有模块导入。
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

首先，重新加载由 `1_notmnist.ipynb` 所生成的数据。

In [0]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # 提示帮助 gc 释放内存
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


重新格式化形成一个 shape 以更适用于我们所要训练的模型：
- 数据格式化为一个 flat 矩阵，
- 标签格式化为浮点 1-hot 编码。

In [0]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # 将 0 映射到 [1.0, 0.0, 0.0 ...], 1 映射到 [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


我们首先使用简单的梯度下降训练一个多项逻辑回归。

TensorFlow 会这么做：
* 首先描述下你是希望如何执行计算的：输入、变量和操作会是什么样。这由一个计算图的节点所创建完成。以上所述都包含在以下的块之中：

      with graph.as_default():
          ...

* 然后，你可以通过调用 `session.run()` 根据自己的意愿执行多次图操作，为图返回的结果提供输出。这个运行时间操作均包含在以下块之中：

      with tf.Session(graph=graph) as session:
          ...

把所有的数据加载到 Tensorflow 并根据我们的训练要求构建计算图：

In [0]:
# 使用梯度下降训练，即便这样大小的数据也会成本太高。
# 为了加速周转时间，将训练数据分为子集。
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # 输入数据。
  # 加载训练、验证和测试数据为
  # 附属于图的常量。
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 变量。
  # 这是我们需要训练的参数。 
  # 使用（截断）正态分布为权重矩阵进行随机值赋值。
  # 偏差随机赋值为零。
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # 训练计算。
  # 我们将输入与权重矩阵相乘，然后加上偏差。
  # 我们使用 softmax 和交叉熵进行计算（这是 Tensorflow 里的操作，因为
  # 这十分常见，并能够进行优化）。
  # 我们取所有训练样本的本次交叉熵计算的平均值：这是我们的损失。
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # 优化。
  # 我们将使用梯度下降寻找最小的损失值。
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # 训练、验证和测试数据的预测。
  # 这不是训练的一部分，但至此我们才能得到
  # 我们训练的精确数据。
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

让我们执行计算和迭代：

In [0]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # 这是一个一次性的操作，确保参数已按照我们图所描述的进行初始化操作
  # 权重矩阵随机赋值，
  # 偏差为零。
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # 执行计算。我们使用 .run() 需要执行优化，
    # 然后获取 numpy 数组形式的损失值和
    # 训练预测值。
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # 对 valid_prediction 调用 .eval() 基本上如同调用 run()，
      # 但只是得到一个 numpy 数组。注意它会重新计算所有
      # 与其存在依赖关系的图。
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0 : 17.2939
Training accuracy: 10.8%
Validation accuracy: 13.8%
Loss at step 100 : 2.26903
Training accuracy: 72.3%
Validation accuracy: 71.6%
Loss at step 200 : 1.84895
Training accuracy: 74.9%
Validation accuracy: 73.9%
Loss at step 300 : 1.60701
Training accuracy: 76.0%
Validation accuracy: 74.5%
Loss at step 400 : 1.43912
Training accuracy: 76.8%
Validation accuracy: 74.8%
Loss at step 500 : 1.31349
Training accuracy: 77.5%
Validation accuracy: 75.0%
Loss at step 600 : 1.21501
Training accuracy: 78.1%
Validation accuracy: 75.4%
Loss at step 700 : 1.13515
Training accuracy: 78.6%
Validation accuracy: 75.4%
Loss at step 800 : 1.0687
Training accuracy: 79.2%
Validation accuracy: 75.6%
Test accuracy: 82.9%


现在，让我们转向随机梯度下降训练，它更加快速。

图基本是相似的，除了所有训练数据都在一个常量节点上，我们创建一个 `Placeholder` 节点，在每次调用 `session.run()` 时传输实际数据。

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # 输入数据。对于训练数据，我们使用一个 placeholder ，
  # 在运行时，传输一个训练的 minibatch 。
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 变量。
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # 训练计算。
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # 优化。
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # 训练、验证和测试数据的预测。
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

让我们运行起来：

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # 在已经过随机化处理的训练数据里选取一个 offset。
    # 注意：每次迭代时我们可以使用更优的随机化处理数据。
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # 生成一个 minibatch。
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # 准备一个字典指定 session 从哪里获取 minibatch。
    # 字典的键指定图的 placeholder 节点，
    # 值是由 numpy 数组所 feed 的数据。
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 16.8091
Minibatch accuracy: 12.5%
Validation accuracy: 14.0%
Minibatch loss at step 500 : 1.75256
Minibatch accuracy: 77.3%
Validation accuracy: 75.0%
Minibatch loss at step 1000 : 1.32283
Minibatch accuracy: 77.3%
Validation accuracy: 76.6%
Minibatch loss at step 1500 : 0.944533
Minibatch accuracy: 83.6%
Validation accuracy: 76.5%
Minibatch loss at step 2000 : 1.03795
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2500 : 1.10219
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 3000 : 0.758874
Minibatch accuracy: 82.8%
Validation accuracy: 78.8%
Test accuracy: 86.1%


---
问题
-------

使用 SGD 将逻辑回归示例变为带有一层激活函数隐层和 1024 个节点的神经网络 [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu)。这个模型可以改进你的验证集/测试集准确度。

---